# PHQ-9 Depression Screening Analysis

This notebook implements the PHQ-9 depression screening questionnaire with data storage functionality.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import sqlite3
import json

In [ ]:
# PHQ-9 Questions
questions = [
    "Little interest or pleasure in doing things?",
    "Feeling down, depressed, or hopeless?",
    "Trouble falling or staying asleep, or sleeping too much?",
    "Feeling tired or having little energy?",
    "Poor appetite or overeating?",
    "Feeling bad about yourself - or that you are a failure?",
    "Trouble concentrating on things?",
    "Moving or speaking slowly or being fidgety/restless?",
    "Thoughts that you would be better off dead or of hurting yourself?"
]

# Answer options and their scores
options = {
    0: "Not at all",
    1: "Several days",
    2: "More than half the days",
    3: "Nearly every day"
}

In [ ]:
def create_database():
    """Create SQLite database to store assessment results"""
    conn = sqlite3.connect('depression_screening.db')
    c = conn.cursor()
    
    c.execute('''
        CREATE TABLE IF NOT EXISTS assessments (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            answers TEXT,
            total_score INTEGER,
            severity TEXT
        )
    ''')
    
    conn.commit()
    conn.close()

# Create database
create_database()

In [ ]:
def get_severity(score):
    """Determine depression severity based on total score"""
    if score <= 4:
        return "None-minimal"
    elif score <= 9:
        return "Mild"
    elif score <= 14:
        return "Moderate"
    elif score <= 19:
        return "Moderately Severe"
    else:
        return "Severe"

def get_recommendation(severity):
    """Get treatment recommendation based on severity"""
    recommendations = {
        "None-minimal": "No treatment may be needed",
        "Mild": "Watchful waiting; repeat PHQ-9 at follow-up",
        "Moderate": "Treatment plan, considering counseling, follow-up and/or pharmacotherapy",
        "Moderately Severe": "Active treatment with pharmacotherapy and/or psychotherapy",
        "Severe": "Immediate initiation of pharmacotherapy and expedited referral to mental health specialist"
    }
    return recommendations[severity]

In [ ]:
def conduct_assessment():
    """Conduct the PHQ-9 assessment and store results"""
    answers = []
    
    print("PHQ-9 Depression Screening Questionnaire")
    print("\nOver the last 2 weeks, how often have you been bothered by any of the following problems?")
    print("\nScoring:")
    for score, label in options.items():
        print(f"{score} = {label}")
    
    # Collect answers
    for i, question in enumerate(questions, 1):
        while True:
            try:
                answer = int(input(f"\n{i}. {question}\nYour answer (0-3): "))
                if answer in options:
                    answers.append(answer)
                    break
                else:
                    print("Please enter a valid score (0-3)")
            except ValueError:
                print("Please enter a valid number")
    
    # Calculate results
    total_score = sum(answers)
    severity = get_severity(total_score)
    recommendation = get_recommendation(severity)
    
    # Store results
    conn = sqlite3.connect('depression_screening.db')
    c = conn.cursor()
    c.execute(
        "INSERT INTO assessments (timestamp, answers, total_score, severity) VALUES (?, ?, ?, ?)",
        (datetime.now().isoformat(), json.dumps(answers), total_score, severity)
    )
    conn.commit()
    conn.close()
    
    # Display results
    print(f"\nResults:")
    print(f"Total Score: {total_score}")
    print(f"Depression Severity: {severity}")
    print(f"Recommendation: {recommendation}")
    
    return total_score, severity, recommendation

In [ ]:
def view_history():
    """View assessment history"""
    conn = sqlite3.connect('depression_screening.db')
    df = pd.read_sql_query("SELECT * FROM assessments", conn)
    conn.close()
    
    if len(df) == 0:
        print("No assessment history found.")
        return
    
    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Parse answers from JSON
    df['answers'] = df['answers'].apply(json.loads)
    
    print("\nAssessment History:")
    for _, row in df.iterrows():
        print(f"\nDate: {row['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Score: {row['total_score']}")
        print(f"Severity: {row['severity']}")
        
    return df

In [ ]:
# Run an assessment
total_score, severity, recommendation = conduct_assessment()

In [ ]:
# View assessment history
history_df = view_history()